# Imports

In [36]:
%load_ext autoreload
%autoreload 2
import sys
from nlppen.extraccion.utils.Txt2Numbers import Txt2Numbers
from nlppen.analisis import Analisis
from nlppen.seleccion import Seleccion
from nlppen.spark_udfs import solo_portanto, solo_considerando, solo_resultando, spark_get_spacy
from nlppen.sentencias_estructurales import SentenciasEstructurales
from pyspark.sql import SparkSession
from pyspark.sql.functions import length

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Initialize spark context

In [37]:
spark = (SparkSession
         .builder
         .appName("Transforming sentences")
         .config("spark.num.executors", "8")
         .config("spark.executor.memory", "10g")
         .config("spark.executor.cores", "6")
         .config("spark.driver.memory", "10g")
         .config("spark.memory.offHeap.enabled", True)
         .config("spark.memory.offHeap.size", "25g")
         .config("spark.sql.execution.arrow.pyspark.enabled", "true")
         .getOrCreate())

sc = spark.sparkContext
sc.uiWebUrl

'http://7416269fd645:4040'

# Buscar terminos en la sección de por lo tanto de la sentencia y aplicar filtro del dataset

In [38]:
terminos = {
    'seguimiento': [r'\bseguimiento\b'],
    'se ordena': [r'\bse ordena\b'],
    'plan': [r'\bplan\b'],
    'plazo': [r'\bplazo\b']
}
seleccion = Seleccion(terminos, spark, parquet_path='./dataset', datasets_path='./datasets/estructurales')
print("Cantidad elementos originales : " + str(seleccion.sdf.count()))
seleccion.filtrar_sentencias(preprocess=solo_portanto, keepRowEmpty=True)
print("Cantidad elementos despues de filtrados : " + str(seleccion.sdf.count()))
estructurales = SentenciasEstructurales(seleccion)

Cantidad elementos originales : 1365


Cantidad elementos despues de filtrados : 1365


# Formar dataset de sentencias estructurales

In [39]:
from pyspark.sql.types import *

columnas = {
    'se ordena PER' : ArrayType(StringType()),
    'se ordena LOC' : ArrayType(StringType()),
    'se ordena ORG' : ArrayType(StringType()),
    'se ordena MISC' : ArrayType(StringType()),
    'se ordena GPE' : ArrayType(StringType()),
    'se ordena Ent Pub' : ArrayType(StringType())
}
estructurales.separarSeOrdena(columnas, True, True)

columnas = {
    'extension sentencia' : IntegerType(),
    'extension por lo tanto' : IntegerType()
}
estructurales.extraerExtension(columnas, True)

columnas = {
    'plazosDefinidos' : ArrayType(TimestampType())
}
estructurales.plazosDefinidos(columnas, True)

columnas = {
    'FechaSolicitud' : TimestampType(),
}

estructurales.extrarFechaRecibido(columnas, True)
#Sobreescribir el dataset de filtro de sentencias con las nuevas columnas
estructurales.seleccion.guardarDatos()


# Cargar procesamiento de sentencias estructurales.

In [15]:
terminos = {
    'seguimiento': [r'\bseguimiento\b'],
    'se ordena': [r'\bse ordena\b'],
    'plan': [r'\bplan\b'],
    'plazo': [r'\bplazo\b']
}
seleccion = Seleccion(terminos, spark, parquet_path='./dataset', datasets_path='./datasets/estructurales')
seleccion.cargarPreprocesados()

estructurales = SentenciasEstructurales(seleccion)
s = estructurales.seleccion.sdf
s.limit(10).toPandas()

,archivo,txt,tema,tema_prob,expediente,fechahora_ext,redactor_ext,tipoResolucion_ext,termino_ext,voto_salvado,...,Ley_ents,EntidadPublica_ents,Constitución_ents,Fecha_ents,anno,seguimiento,se_ordena,plan,plazo,FechaSolicitud
0,784F6.html,*050151950007CO* Exp: 05-015195-0007-CO Res. N...,PETICION,0.086251,291C05,2006-05-30 17:14:00,Armijo Sancho,Recurso de Amparo,Con lugar parcial,False,...,None,None,None,None,2006,0,1,0,0,2005-11-24 10:05:00
1,72C2E.html,*060010870007CO* Exp: 06-001087-0007-CO Res. N...,TRABAJO,0.451749,060010870007CO,2006-02-22 14:44:00,Vargas Benavides,Recurso de Amparo,Con lugar,False,...,None,None,None,None,2006,0,1,0,1,2006-01-31 12:27:00
2,7865A.html,*060032250007CO* Exp: 06-003225-0007-CO Res. N...,PENITENCIARIO,0.335066,060032250007CO,2006-07-19 19:41:00,Cruz Castro,Hábeas Corpus,Con lugar,False,...,"[Ley de la Jurisdicción Constitucional, Ley de...",None,None,"[28 de abril del 2005, 31 de marzo del 2006, 1...",2006,0,0,0,0,2005-04-28 14:13:00
3,79A74.html,*060096310007CO* Exp: 06-009631-0007-CO Res. N...,PETICION,0.252051,060096310007CO,2006-08-23 16:26:00,Calzada Miranda,Recurso de Amparo,Con lugar,False,...,None,None,"[27, 27, 27]","[7 de agosto del 2006, 9 de mayo del 2006, 9 d...",2006,0,0,0,0,2006-08-07 14:05:00
4,72D53.html,*050165730007CO* Exp: 05-016573-0007-CO Res: 2...,TRABAJO,0.445111,050165730007CO,2006-03-10 13:10:00,Cruz Castro,Recurso de Amparo,Con lugar,False,...,None,"[Ministerio de Educación Pública, Ministerio d...",[41],"[23 de diciembre del 2005, 17 de noviembre del...",2006,0,0,0,0,2005-12-23 14:20:00
5,7B98B.html,*060035870007CO* Exp: 06-003587-0007-CO Res. N...,PETICION,0.086251,060035870007CO,NaT,Sosto López,Recurso de Amparo,Con lugar,False,...,"[Ley de la Jurisdicción Constitucional, Ley de...","[Ministerio de Obras Públicas y Transportes, M...","[50, 21]","[28 de marzo del 2006, 2 de mayo del 2004, 24 ...",2006,0,1,0,1,2006-03-28 09:41:00
6,7DA84.html,*060142170007CO* Exp: 06-014217-0007-CO Res. N...,TRABAJO,0.644294,060142170007CO,2006-12-12 17:22:00,Jinesta Lobo,Recurso de Amparo,Con lugar,False,...,"[Ley General de la Administración Pública, Ley...","[Ministerio de Trabajo y Seguridad Social, Min...","[41, 41, 41, 41, 41]","[20 de noviembre del 2006, 11 de octubre de 20...",2006,0,1,0,1,2006-11-20 11:00:00
7,7544C.html,*060045980007CO* Exp: 06-004598-0007-CO Res. N...,TRABAJO,0.371132,060045980007CO,2006-05-16 15:54:00,Armijo Sancho,Recurso de Amparo,Con lugar,False,...,None,"[Caja Costarricense del Seguro Social, Caja Co...",[41],"[24 de abril de 2006, 22 de mayo de 2002, 23 d...",2006,0,0,0,0,2006-04-24 13:06:00
8,715B6.html,*050147230007CO* Exp: 05-014723-0007-CO Res. N...,PETICION,0.273704,050147230007CO,2006-02-10 11:51:00,Rodríguez Arroyo,Recurso de Amparo,Con lugar,False,...,"[Ley de Armas, Ley de Armas]","[Ministerio de Seguridad Pública, Ministerio d...","[28, 27]",None,2006,0,1,0,0,2005-11-15 08:52:00
9,6FF2A.html,*050143170007CO* Exp: 05-014317-0007-CO Res. N...,PENSION,0.680464,050143170007CO,2006-01-27 12:25:00,Molina Quesada,Recurso de Amparo,Con lugar,False,...,"[Ley de la Jurisdicción Constitucional que, Le...",None,"[27, 41]","[7 de noviembre de 2005, 7 de abril de 2005, 7...",2006,0,1,0,1,2005-11-07 08:10:00


# Zona de trabajo (ignorar)

In [19]:
from pyspark.sql.types import *
columnas = {
    'FechaSolicitud' : TimestampType(),
}
s = estructurales.extrarFechaRecibido(columnas)
s.toPandas()

,archivo,txt,tema,tema_prob,expediente,fechahora_ext,redactor_ext,tipoResolucion_ext,termino_ext,voto_salvado,...,Ley_ents,EntidadPublica_ents,Constitución_ents,Fecha_ents,anno,seguimiento,se_ordena,plan,plazo,FechaSolicitud
0,784F6.html,*050151950007CO* Exp: 05-015195-0007-CO Res. N...,PETICION,0.086251,291C05,2006-05-30 17:14:00,Armijo Sancho,Recurso de Amparo,Con lugar parcial,False,...,None,None,None,None,2006,0,1,0,0,2005-11-24 10:05:00
1,72C2E.html,*060010870007CO* Exp: 06-001087-0007-CO Res. N...,TRABAJO,0.451749,060010870007CO,2006-02-22 14:44:00,Vargas Benavides,Recurso de Amparo,Con lugar,False,...,None,None,None,None,2006,0,1,0,1,2006-01-31 12:27:00
2,7B98B.html,*060035870007CO* Exp: 06-003587-0007-CO Res. N...,PETICION,0.086251,060035870007CO,NaT,Sosto López,Recurso de Amparo,Con lugar,False,...,"[Ley de la Jurisdicción Constitucional, Ley de...","[Ministerio de Obras Públicas y Transportes, M...","[50, 21]","[28 de marzo del 2006, 2 de mayo del 2004, 24 ...",2006,0,1,0,1,2006-03-28 09:41:00
3,7DA84.html,*060142170007CO* Exp: 06-014217-0007-CO Res. N...,TRABAJO,0.644294,060142170007CO,2006-12-12 17:22:00,Jinesta Lobo,Recurso de Amparo,Con lugar,False,...,"[Ley General de la Administración Pública, Ley...","[Ministerio de Trabajo y Seguridad Social, Min...","[41, 41, 41, 41, 41]","[20 de noviembre del 2006, 11 de octubre de 20...",2006,0,1,0,1,2006-11-20 11:00:00
4,715B6.html,*050147230007CO* Exp: 05-014723-0007-CO Res. N...,PETICION,0.273704,050147230007CO,2006-02-10 11:51:00,Rodríguez Arroyo,Recurso de Amparo,Con lugar,False,...,"[Ley de Armas, Ley de Armas]","[Ministerio de Seguridad Pública, Ministerio d...","[28, 27]",None,2006,0,1,0,0,2005-11-15 08:52:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
903,192e6.html,Exp. N.° 6015-S-95 Voto N.° 6701-95...,PETICION,0.238965,6015S95,1995-12-07 15:36:00,Albertazzi Herrera,Recurso de Amparo,Con lugar,False,...,[Ley de la Jurisdicción Constitucio-],"[Poder Ejecutivo, Contraloría General de la Re...",None,"[25 de agosto de 1995, 18 de octubre de 1995]",1995,0,1,0,0,NaT
904,1dd1a.html,Exp N.° 2743-S-93 Voto N.° 3668-93 SALA ...,PENAL,0.999625,2743S93,1993-07-30 08:40:00,Solano Carrera,Hábeas Corpus,Con lugar,False,...,None,None,None,None,1993,0,1,0,0,NaT
905,1d983.html,VOTO N 1923-92.- SALA CONSTITUCIONAL DE LA COR...,PENAL,0.762379,2201B92,1992-07-22 14:36:00,Baudrit- y,Hábeas Corpus,Con lugar,False,...,None,None,None,None,1992,0,0,0,1,NaT
906,159b9.html,RECURSO DE AMPARO Nº 1934-A-90 AGENCIAS KABA...,PETICION,0.086251,1934A90,1992-06-03 16:09:00,Sancho González,Recurso de Amparo,Con lugar,False,...,None,None,None,None,1992,0,0,0,2,NaT


In [20]:
print("Nulos : ", str(s.filter(s.FechaSolicitud.isNull()).count()))
print("Accuracy : " ,s.filter(s.FechaSolicitud.isNotNull()).count()/s.count())

#s = s.filter(s.FechaSolicitud.isNull())
#s = estructurales.seleccion.sdf

Nuevo 237
Accuracy : 0.8263736263736263


In [26]:
s.filter(s.FechaSolicitud.isNull()).groupby(s.anno).count().toPandas().sort_values('anno')

,anno,count
0,1990,3
11,1991,6
19,1992,6
28,1993,7
7,1994,6
17,1995,4
12,1996,13
6,1997,17
13,1998,25
26,1999,7


In [58]:
s.filter(s.FechaSolicitud.isNotNull()).groupby(s.anno).count().toPandas().sort_values('anno')

,anno,count
7,1994,3
11,1996,2
6,1997,12
12,1998,2
23,1999,4
18,2000,20
16,2001,29
24,2002,33
0,2003,44
10,2004,28


In [7]:
txt = "Mediante escrito presentado ante la Secretaría de esta Sala a las 19:00 horas del 23 de enero de 2015, el recurrido,"
txt2 = str(txt)
nlp = spark_get_spacy('es_core_news_lg')
doc = nlp(txt)
for token in doc:
        print(token.text, token.pos_)

Mediante ADP
escrito NOUN
presentado ADJ
ante ADP
la DET
Secretaría PROPN
de ADP
esta DET
Sala PROPN
a ADP
las DET
19:00 NOUN
horas NOUN
del ADP
23 NUM
de ADP
enero NOUN
de ADP
2015 NUM
, PUNCT
el DET
recurrido NOUN
, PUNCT


In [22]:
#txt = "Dos mil dieciocho"
txt = "Por escrito recibido en la Secretaría de la Sala a las once horas cuarenta y cuatro minutos del primero de agosto en curso, la recurrente interpone recurso de amparo contra el Director de Recursos Humanos y la Jefe de la Unidad de Control de Pagos, ambos del Ministerio de Educación Pública y manifiesta que es funcionaria de ese ministerio."
print(re.search("(año en curso)|(presente año)|(en curso)", txt.lower()))

<re.Match object; span=(114, 122), match='en curso'>
